# [순환신경망 RNN]<hr>

In [2]:
import torch
import torch.nn as nn

In [3]:
# 데이터 초기 Hidden state
input = torch.randn(1,3,10) # 입력 데이터( 배치크기, 시퀀스길이, 피쳐길이(input size) )
h0 = torch.randn(1,3,1)     # 히든 초기값 사이즈

rnn = nn.RNN(input_size=10, hidden_size=1, num_layers=1)

ouput, hn = rnn(input, h0)

In [4]:
# 설계 : 다층 RNN 층 2개
# => input
# 입력 초기 텐서들                                    # 패딩할 때, 지정했던 길이 = 시퀀스 길이  # 피쳐길이 : 원핫인코딩하고 임베딩한 피쳐의 개수
input = torch.randn(1,4,10) # 입력 데이터( 배치크기, 시퀀스길이(1개 문장구성 단어 수), 피쳐길이(1개 단어 표현하는 수 / input size)(중복제외 단어갯수))
# 다섯 문장 줄거고, 하나의 문장은 세개의 단어로 구성되어 있다.
# ex) 나 소년 이다 -> 시퀀스 : 3, 피쳐수 : 10(one-hot encoding으로 바꾸고 임베딩(밀집행렬로 바꾼 것)한 피쳐의 수)
h0 = torch.randn(1,1,5)      # 히든 스테이트 초기화( 양방향인지 단방향인지/ (양방향 유무 +1)* 층수, 배치크기, 히든갯수) 
# 히든 초기값 사이즈 ( 층 개수 *(int(bidirectional)+1), 배치크기, hidden state 개수(ouput size) )

# RNN 인스턴스
rnn = nn.RNN(input_size=10, hidden_size=5, num_layers=1, batch_first=True) # hidden_size = ouput size
# batch_first = True -> batch_size가 맨 앞으로만 가도록 해주면 됨
# batch_first = False→ (seq, batch, feature)
# batch_first = True → (batch, seq, feature)

# 출력 텐서들
output, hn = rnn(input, h0)

In [5]:
from torchinfo import summary

summary(rnn)

Layer (type:depth-idx)                   Param #
RNN                                      85
Total params: 85
Trainable params: 85
Non-trainable params: 0

In [6]:
# rnn 모델의 속성 출력
print(f'[all_weights] : {len(rnn.all_weights)}')
print(rnn.all_weights)

[all_weights] : 1
[[Parameter containing:
tensor([[ 0.3088,  0.2480, -0.4184, -0.3413,  0.4081, -0.3907, -0.2753, -0.3320,
         -0.0839,  0.2592],
        [-0.4429, -0.1063,  0.3559,  0.1028, -0.1213, -0.3576,  0.4148,  0.4286,
         -0.1442,  0.2364],
        [ 0.4278,  0.0182, -0.1802, -0.2281,  0.4253, -0.4291,  0.1422, -0.3793,
         -0.3880,  0.2124],
        [-0.4347,  0.0464,  0.0720, -0.0533,  0.2318, -0.3767,  0.1195,  0.3837,
          0.0946, -0.4399],
        [-0.1703,  0.1163, -0.1654, -0.4054, -0.2154,  0.3834,  0.3050, -0.3497,
          0.3849,  0.3472]], requires_grad=True), Parameter containing:
tensor([[ 0.4195, -0.2977, -0.3780,  0.1948,  0.4381],
        [-0.2065,  0.3127, -0.2177, -0.3882, -0.0465],
        [-0.3705,  0.1920, -0.1972, -0.1990, -0.2406],
        [ 0.3114,  0.3571, -0.0555,  0.4432, -0.2294],
        [-0.3858,  0.0808,  0.3727,  0.2277,  0.3042]], requires_grad=True), Parameter containing:
tensor([-0.2826, -0.0467, -0.1793,  0.2959, -0.096

In [7]:
for name, param in rnn.named_parameters():
    print(f'[{name}] \n{param}]') 
# 각각 배치마다 가중치 5개 ( input(10개)에 hidden(5개) 총 50개 / hh(5개))(층이 두개라면 5개가 다음 입력층의 개수) / 각각 절편 5개( ih(input) , hh(heddin) )
# hidden state는 하나의 시퀀스를 처리해야함 -> 시퀀스길이의 값이 배치크기가 되어야함
    
# [weight_ih_l0] : 각 피쳐의 가중치 / 10개고 히든스테이트가 5개니까 총 50개
# [weight_hh_l0] : 각 히든 스테이트의 가중치 / 5개고 히든스테이트가 5개니까 총 25개
# [bias] : 각 절편 
    
# 셀이 퍼셉트론이나 커널과 같다
# 히든스테이트는 이전의 결과값
# 처음의 히든스테이트는 0
    
# 결과값은 AF를 거쳐서 결과값으로 내보냄, AF 안 거치고 다시 히든 스테이트에도 보관

[weight_ih_l0] 
Parameter containing:
tensor([[ 0.3088,  0.2480, -0.4184, -0.3413,  0.4081, -0.3907, -0.2753, -0.3320,
         -0.0839,  0.2592],
        [-0.4429, -0.1063,  0.3559,  0.1028, -0.1213, -0.3576,  0.4148,  0.4286,
         -0.1442,  0.2364],
        [ 0.4278,  0.0182, -0.1802, -0.2281,  0.4253, -0.4291,  0.1422, -0.3793,
         -0.3880,  0.2124],
        [-0.4347,  0.0464,  0.0720, -0.0533,  0.2318, -0.3767,  0.1195,  0.3837,
          0.0946, -0.4399],
        [-0.1703,  0.1163, -0.1654, -0.4054, -0.2154,  0.3834,  0.3050, -0.3497,
          0.3849,  0.3472]], requires_grad=True)]
[weight_hh_l0] 
Parameter containing:
tensor([[ 0.4195, -0.2977, -0.3780,  0.1948,  0.4381],
        [-0.2065,  0.3127, -0.2177, -0.3882, -0.0465],
        [-0.3705,  0.1920, -0.1972, -0.1990, -0.2406],
        [ 0.3114,  0.3571, -0.0555,  0.4432, -0.2294],
        [-0.3858,  0.0808,  0.3727,  0.2277,  0.3042]], requires_grad=True)]
[bias_ih_l0] 
Parameter containing:
tensor([-0.2826, -0.0467

In [53]:
# RNN 출력 텐서 output
output.shape, output.ndim

(torch.Size([1, 4, 5]), 3)